In [ ]:
import os

import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from tqdm import tqdm

In [ ]:
data_dir = kagglehub.dataset_download(
    "siddhantmaji/unified-waste-classification-dataset"
)

print("Path to dataset files:", data_dir)

100%|██████████| 955M/955M [00:11<00:00, 90.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/siddhantmaji/unified-waste-classification-dataset/versions/1


In [ ]:
data_dir = f"{data_dir}/content/unified_dataset"
BATCH_SIZE = 32
IMG_SIZE = 224

# Preprocessing transforms only, no data augmentation
transform = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Load full dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Stratified train/val split
targets = [sample[1] for sample in dataset.samples]
train_idx, val_idx = train_test_split(
    range(len(dataset)), test_size=0.2, stratify=targets, random_state=42
)

train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v3_large(pretrained=True)
# Replace classifier's last layer to match number of classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 8)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 87.0MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(
        f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader):.4f} | Accuracy = {acc:.2f}%"
    )

Epoch 1/10: 100%|██████████| 1600/1600 [03:40<00:00,  7.25it/s]


Epoch 1: Loss = 0.3533 | Accuracy = 88.41%


Epoch 2/10: 100%|██████████| 1600/1600 [03:38<00:00,  7.33it/s]


Epoch 2: Loss = 0.1054 | Accuracy = 96.45%


Epoch 3/10: 100%|██████████| 1600/1600 [03:37<00:00,  7.36it/s]


...

Epoch 8: Loss = 0.0241 | Accuracy = 99.23%


Epoch 9/10: 100%|██████████| 1600/1600 [03:37<00:00,  7.37it/s]


Epoch 9: Loss = 0.0202 | Accuracy = 99.33%


Epoch 10/10: 100%|██████████| 1600/1600 [03:38<00:00,  7.31it/s]

Epoch 10: Loss = 0.0203 | Accuracy = 99.34%


In [ ]:
model.eval()
val_loss, correct, total = 0.0, 0, 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}% | Loss: {val_loss/len(val_loader):.4f}")

Validation Accuracy: 97.98% | Loss: 0.0856


In [ ]:
torch.save(model, "../models/mobilenetv3_garbage_classifier_full.pt")

In [ ]:
# from google.colab import files
# files.download("/content/mobilenetv3_garbage_classifier_full.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>